In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain import hub ## For defining prompts
from langchain.agents import create_openai_functions_agent##Creates agents from openai
from langchain_community.tools.tavily_search import TavilySearchResults ## Tavily search results
from langchain_openai.chat_models import ChatOpenAI

### Defining Lang chain agent
* We will define langchain agent with tools available.
    * Here the tools are Tavilysearch and vector DB

In [3]:
##Create Vector DB
from langchain_community.document_loaders import WebBaseLoader # Loads data from a web page
from langchain_text_splitters import RecursiveCharacterTextSplitter # Split the characters into chunks
from langchain_community.vectorstores import FAISS # ccreates a vector store
from langchain_openai import OpenAIEmbeddings # Embedding of text


loader= WebBaseLoader("https://docs.smith.langchain.com/overview")
docs= loader.load()
documents= RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
).split_documents(docs)

vector= FAISS.from_documents(documents=documents, embedding=OpenAIEmbeddings())
retriever= vector.as_retriever()

##3 Tavily Search
search= TavilySearchResults(max_results=1)

#### Create retriever tool for retriever

In [4]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool= create_retriever_tool(
    retriever=retriever,
    name= "langsmith_retriever",
    description="Provides information about langsmith and its components"
)

### Combine all the tools

In [5]:
tools= [search, retriever_tool]

#### Create an agent now

In [6]:
from langchain_openai import ChatOpenAI
from langchain import hub

## Pull prompt
prompt= hub.pull("hwchase17/openai-functions-agent")

### Choose LLm to drive agent
llm= ChatOpenAI(model="gpt-3.5-turbo", temperature=0, streaming=True)

# As we are using openai we will construct agent with openai
agent_runnable= create_openai_functions_agent(llm, tools, prompt)

#### Defining graph state
Generally a default/basic/traditional langchain agent has below attributes
1. input: This is the query/information that we will get from user
2. chat_history: This stores history of the chat
3. Intermediate_steps: The list of actions and corresponding observations that agen consider over time. This is updated at each iteration of the agent.
4. agent_outcome: This is response from the agent after going through nodes via edges. This is a response either from AgentAction and AgentFinish. AgentExecutor should finish when it is AgentFinish otherwise it should call requested tool.

* Define Schema for Agent State

In [7]:
import operator
from typing import Annotated, TypedDict, Union
## Define AgentACtion and AgentFinish
from langchain_core.agents import AgentAction,AgentFinish
from langchain_core.messages import BaseMessage

##Define schema class 
class AgentState(TypedDict):
    input: str
    chat_history: list[BaseMessage]
    ## Beside AgentAction an AgentFinish it can return None
    agent_outcome: Union[AgentAction, AgentFinish, None]
    ###We will mention that Corresponding action should be added to 
    ### intermediate steps and this is via oeprator.add
    intermediate_steps: Annotated[list[tuple[AgentAction, str]], operator.add]

## Define nodes
* We define nodes in the graph. Nodes can be Runnables or a function that can return customized result.
* There are two main nodes that we definetly need.
    * The agent: This is responsible to decide which action to take
    * function to invoke tools: IF agent decides to take action, for that we define function what to execute

* We define edgees which give information of the proces flow. It defines where to flow and which function to execute.
    * Conditional edge: With If else
    * Normal edge: Normal straight edge

In [8]:
from langchain_core.agents import AgentFinish
## Define tool executor
from langgraph.prebuilt.tool_executor import ToolExecutor
## This is a helper wheere it takes all the details of tools.
## and invokes which ever tool we want when we pass respective tool.
tool_executor= ToolExecutor(tools)

## Define Agent
def run_agent(data):
    ## This is the initial step
    ## Hence we invoke the data
    agent_outcome= agent_runnable.invoke(data)
    return {"agent_outcome": agent_outcome}

## Once we run agent then it will return tool name to run
## Create fuction to execute that tool name
def execute_tool(data):
    ## agent_outcome will return agent action, agentEnd as defined in schema
    ## We will choose the latest one
    agent_action= data['agent_outcome']
    output= tool_executor.invoke(agent_action)
    ## return it to intermediate steps which stores all the details
    return {"intermediate_steps": [(agent_action, str(output))]}

## Defining logic weather the process has to continue or end based on process flow
def should_continue(data):
    ##Check the outcome variable an decide whether to continue or to end the flow
    if isinstance(data['agent_outcome'], AgentFinish):
        return "end"
    else:
        return "continue"

### Now its time to define graph
* We will pull all the information that we have designed to crate graph

In [9]:
from langgraph.graph import END, StateGraph, START

## Define State graph
workflow= StateGraph(AgentState)

## Define initial two nodes that play with in the middle of the process
workflow.add_node("agent", run_agent)
workflow.add_node("action", execute_tool)

## Define edges where the node has to start
workflow.add_edge(START, "agent")

## WE will now add conditional edge
workflow.add_conditional_edges(
    
    "agent",
    
    should_continue,
    
    ## IF above should-continue returns below values
    ## The keys are strings and values are nodes in the below result
    {
        # IF the response is tools then we continue 
        "continue": "action",
        # else
        "end": END,
    }
    
)

## Once tools are called from the conditional edge
## Then we will call agent node again to invoke.
workflow.add_edge("action", "agent")

## Once we get the flow we will compile it.
## This will be compiled to a langchainrunnable
## This can be used as any other general runnables
app= workflow.compile()


In [12]:
inputs= {"input": "what is weather in SF?", "chat_history": []}

from pprint import pprint
for result in app.stream(inputs):
    pprint(list(result.values())[0])
    pprint("-----")

{'agent_outcome': AgentActionMessageLog(tool='tavily_search_results_json', tool_input={'query': 'weather in San Francisco'}, log="\nInvoking: `tavily_search_results_json` with `{'query': 'weather in San Francisco'}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"query":"weather in San Francisco"}', 'name': 'tavily_search_results_json'}}, response_metadata={'finish_reason': 'function_call', 'model_name': 'gpt-3.5-turbo-0125'}, id='run-26e4e90a-7239-45ff-b4b9-4eb98c8fad0f-0')])}
'-----'
{'intermediate_steps': [(AgentActionMessageLog(tool='tavily_search_results_json', tool_input={'query': 'weather in San Francisco'}, log="\nInvoking: `tavily_search_results_json` with `{'query': 'weather in San Francisco'}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"query":"weather in San Francisco"}', 'name': 'tavily_search_results_json'}}, response_metadata={'finish_reason': 'function_call', 'model_name'

In [13]:
search.invoke("What is weather in SF?")

[{'url': 'https://www.weatherapi.com/',
  'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.78, 'lon': -122.42, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1722579213, 'localtime': '2024-08-01 23:13'}, 'current': {'last_updated_epoch': 1722578400, 'last_updated': '2024-08-01 23:00', 'temp_c': 14.7, 'temp_f': 58.4, 'is_day': 0, 'condition': {'text': 'Partly Cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/night/116.png', 'code': 1003}, 'wind_mph': 5.8, 'wind_kph': 9.4, 'wind_degree': 245, 'wind_dir': 'WSW', 'pressure_mb': 1018.0, 'pressure_in': 30.06, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 92, 'cloud': 45, 'feelslike_c': 14.3, 'feelslike_f': 57.8, 'windchill_c': 14.3, 'windchill_f': 57.8, 'heatindex_c': 14.7, 'heatindex_f': 58.4, 'dewpoint_c': 13.3, 'dewpoint_f': 55.9, 'vis_km': 10.0, 'vis_miles': 6.0, 'uv': 1.0, 'gust_mph': 9.1, 'gust_kph': 14.7}}"}]

In [15]:
inputs= {"input": "How to create dataset in langsmith?", "chat_history": []}

from pprint import pprint
for result in app.stream(inputs):
    pprint(list(result.values())[0])
    pprint("-----")

[{'agent_outcome': AgentActionMessageLog(tool='langsmith_retriever', tool_input={'query': 'How to create dataset in langsmith?'}, log="\nInvoking: `langsmith_retriever` with `{'query': 'How to create dataset in langsmith?'}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"query":"How to create dataset in langsmith?"}', 'name': 'langsmith_retriever'}}, response_metadata={'finish_reason': 'function_call', 'model_name': 'gpt-3.5-turbo-0125'}, id='run-b8f346e3-9ea5-4502-822f-b6b031aa2a30-0')])}]
'-----'
[{'intermediate_steps': [(AgentActionMessageLog(tool='langsmith_retriever', tool_input={'query': 'How to create dataset in langsmith?'}, log="\nInvoking: `langsmith_retriever` with `{'query': 'How to create dataset in langsmith?'}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"query":"How to create dataset in langsmith?"}', 'name': 'langsmith_retriever'}}, response_metadata={'finish_reason': 'f